# 103 Spark - Movielens

The goal of this lab is to run some analysis on a different dataset, [MovieLens](https://grouplens.org/datasets/movielens/).

- Scala
    - [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
    - [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
    - [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)
- Python
    - [Spark programming guide](https://spark.apache.org/docs/3.5.0/rdd-programming-guide.html)
    - [All RDD APIs](https://spark.apache.org/docs/3.5.0/api/python/reference/api/pyspark.RDD.html)

**Download the dataset** from [here](https://big.csr.unibo.it/downloads/bigdata/ml-dataset.zip), unzip it and put it in the ```datasets/big``` folder.

- ml-movies.csv (<u>movieId</u>:Long, title:String, genres:String) 
    - genres are separated by pipelines  (e.g., "comedy|drama|action")
    - each movie is associated with many ratings

- ml-ratings.csv (<u>userId</u>:Long, <u>movieId</u>:Long, rating:Double, year:Int)
    - each rating is associated with many tags
    - ml-ratings-sample.csv is a small sample of ml-ratings.csv, useful for developing
- ml-tags.csv (<u>userId</u>:Long, <u>movieId</u>:Long, <u>tag</u>:String, year:Int) 

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Local Spark") \
    .config('spark.ui.port', '4040') \
    .getOrCreate()
sc = spark.sparkContext

sc

<SparkContext master=local appName=Local Spark>

In [8]:
path_to_datasets = "../../../../datasets/big/"

path_ml_movies = path_to_datasets + "ml-movies.csv"
path_ml_ratings = path_to_datasets + "ml-ratings.csv" # switch to ml-ratings-sample.csv for development, but use the full one for optimization evaluations!
path_ml_tags = path_to_datasets + "ml-tags.csv"

In [9]:
import re
from datetime import datetime
from typing import Optional, Tuple

class MovieLensParser:
    no_genres_listed = "(no genres listed)"
    comma_regex = re.compile(r',(?=(?:[^"]*"[^"]*")*[^"]*$)')
    pipe_regex = re.compile(r'\|(?=(?:[^"]*"[^"]*")*[^"]*$)')
    quotes = '"'

    @staticmethod
    def year_from_timestamp(timestamp: str) -> int:
        """Convert from timestamp (string) to year (int)."""
        ts = int(timestamp.strip())
        return datetime.utcfromtimestamp(ts).year

    @staticmethod
    def parse_movie_line(line: str) -> Optional[Tuple[int, str, str]]:
        """Parse a movie record into (movieId, title, genres)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            title = parts[1].strip()
            if title.startswith(MovieLensParser.quotes):
                title = title[1:]
            if title.endswith(MovieLensParser.quotes):
                title = title[:-1]
            return (int(parts[0].strip()), title, parts[2].strip())
        except Exception:
            return None

    @staticmethod
    def parse_rating_line(line: str) -> Optional[Tuple[int, int, float, int]]:
        """Parse a rating record into (userId, movieId, rating, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return int(parts[0].strip()), int(parts[1].strip()), float(parts[2].strip()), year
        except Exception:
            return None

    @staticmethod
    def parse_tag_line(line: str) -> Optional[Tuple[int, int, str, int]]:
        """Parse a tag record into (userId, movieId, tag, year)."""
        try:
            parts = MovieLensParser.comma_regex.split(line)
            year = MovieLensParser.year_from_timestamp(parts[3])
            return (int(parts[0].strip()), int(parts[1].strip()), parts[2].strip(), year)
        except Exception:
            return None


In [10]:
rddMovies = sc.textFile(path_ml_movies).map(MovieLensParser.parse_movie_line).filter(lambda x: x is not None)
rddRatings = sc.textFile(path_ml_ratings).map(MovieLensParser.parse_rating_line).filter(lambda x: x is not None)
rddTags = sc.textFile(path_ml_tags).map(MovieLensParser.parse_tag_line).filter(lambda x: x is not None)

## 103-1 Datasets exploration

Cache the datasets and answer the following questions:

- How many (distinct) users, movies, ratings, and tags?
- How many (distinct) genres?
- On average, how many ratings per user?
- On average, how many ratings per movie?
- On average, how many genres per movie?
- What is the range of ratings?
- Which years? (print an ordered list)
- On average, how many ratings per year?

In [11]:
rddMoviesCached = rddMovies.cache()
rddRatingsCached = rddRatings.cache()
rddTagsCached = rddTags.cache()

In [12]:
print(f"Number of movies: {rddMoviesCached.count()}")
print(f"Number of ratings: {rddRatingsCached.count()}")
print(f"Number of tags: {rddTags.map(lambda x: x[2]).distinct().count()}")
print(f"Number of users: {rddRatingsCached.map(lambda x: x[0]).distinct().count()}")

Number of movies: 58098
Number of ratings: 1000000
Number of tags: 74702
Number of users: 10073


In [29]:
print("Number of genres: " + str(rddMoviesCached.flatMap(lambda x: x[2].split("|")).distinct().count()))

Number of genres: 20


In [33]:
avgRatPerUser = rddRatingsCached.\
    map(lambda x: (x[0],1)).\
    reduceByKey(lambda x,y: x+y).\
    map(lambda x: (x[1],1)).\
    reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))

print(f"Number of ratings per user: {(avgRatPerUser[0]/avgRatPerUser[1])}")

Number of ratings per user: 99.27529038022436


In [35]:
avgRatPerMovie = rddRatingsCached.\
    map(lambda x: (x[1],1)).\
    reduceByKey(lambda x,y: x+y).\
    map(lambda x: (x[1],1)).\
    reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
print(f"Number of ratings per movie: {(avgRatPerMovie[0]/avgRatPerMovie[1])}")

Number of ratings per movie: 45.39058599246516


In [37]:
avgGenresPerMovie = rddMoviesCached.\
    map(lambda x: (x[0],x[2])).\
    mapValues(lambda x: len(x.split("|"))).\
    reduceByKey(lambda x,y: x+y).\
    map(lambda x: (x[1],1)).\
    reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
print(f"Number of genres per movie: {(avgGenresPerMovie[0]/avgGenresPerMovie[1])}")

Number of genres per movie: 1.8263451409687081


In [38]:
print(f"Range of ratings: {rddRatingsCached.map(lambda x: x[2]).min()} to {rddRatingsCached.map(lambda x: x[2]).max()}")

Range of ratings: 0.5 to 5.0


In [40]:
print(f"Ordered list of years:")
print(sorted(rddRatingsCached.map(lambda x: x[3]).distinct().collect()))

Ordered list of years:
[1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [41]:
avgRatPerYear = rddRatingsCached.\
    map(lambda x: (x[3],1)).\
    reduceByKey(lambda x,y: x+y).\
    map(lambda x: (x[1],1)).\
    reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
print(f"Number of ratings per year: {(avgRatPerYear[0]/avgRatPerYear[1])}")

Number of ratings per year: 43478.260869565216


## 103-2 Compute the average rating for each movie

- Export the result to a file
- Do not start from cached RDDs
- Evaluate:
  - Join-and-Aggregate
  - Aggregate-and-Join
  - Aggregate-and-BroadcastJoin

In [30]:
path_output_avgRatPerMovie = "../../../../output/avgRatPerMovie"
# rdd.coalesce(1).toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

### Join-and-Aggregate

In [41]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],x[2])).\
    join(rddMoviesKV).\
    map(lambda x: ((x[0],x[1][1]),(x[1][0],1)) ).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    map(lambda x: (x[0][0], x[0][1], x[1][0]/x[1][1], x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-Join

In [43]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    join(rddMoviesKV).\
    map(lambda x: (x[0], x[1][1], x[1][0][0], x[1][0][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

### Aggregate-and-BroadcastJoin

In [44]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[1]))
bRddMovies = sc.broadcast(rddMoviesKV.collectAsMap())
avgRatPerMovie = rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    mapValues(lambda x: (x[0]/x[1], x[1])).\
    map(lambda x: (x[0],bRddMovies.value.get(x[0]),x[1][0],x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').option("header", "true").save(path_output_avgRatPerMovie)

Broadcasting does not improve performances in this case, because the join operation in the aggregation-before-join job is done on a limited amount of records, thus the shuffle operation was already quite fast.

## 103-3 Compute the average rating for each genre

Two possible workflows:

1. Pre-aggregation (3 shuffles)

  - Aggregate ratings by movieId
  - Join with movies and map to genres
  - Aggregate by genres
  
2. Join & aggregate (2 shuffles)

  - Join with movies and map to genres
  - Aggregate by genres

In [45]:
path_output_avgRatPerGenre = "../../../../output/avgRatPerGenre"

for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

Which is better?

1. Pre-aggregation (3 shuffles)

  - Aggregate ratings by movieId
    - Input: 724MB (there are 28M ratings, it's ~26B per rating)
    - Output: ~1.5MB (there are 58K movies)
  - Join with movies and map to genres
    - Input: ~1.5MB + 2.7MB (there are 58K movies, it's ~47B per movie)
    - Output: ~6MB (considering 47B per record and 2 genres per movie)
  - Aggregate by genres
    - Input: ~6MB
    - Output: ~1KB (considering 47B per record and that there are 20 genres)
  
2. Join & aggregate (2 shuffles)

  - Join with movies and map to genres
    - Input: 724MB + 2.7MB
      - Actually lower due to serialization and "column pruning"
    - Output: ~1.4GB (considering 47B per record and 2 genres per movie)
      - Actually much lower due to map-side aggregation (i.e., combining) automatically done by Spark
  - Aggregate by genres
    - Input: ~1.4GB
    - Output: ~1KB

Let's verify it by executing them and looking at the execution plans at [localhost:4040](localhost:4040).

In [46]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[2])).flatMapValues(lambda x: x.split("|"))
rddRatings.\
    map(lambda x: (x[1],(x[2],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    join(rddMoviesKV).\
    map(lambda x: (x[1][1],(x[1][0][0],x[1][0][1]))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    map(lambda x: (x[0], x[1][0]/x[1][1], x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').save(path_output_avgRatPerGenre)

In [51]:
rddMoviesKV = rddMovies.map(lambda x: (x[0],x[2])).flatMapValues(lambda x: x.split("|"))
rddRatings.\
    map(lambda x: (x[1],x[2])).\
    join(rddMoviesKV).\
    map(lambda x: (x[1][1],(x[1][0],1))).\
    reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).\
    map(lambda x: (x[0], x[1][0]/x[1][1], x[1][1])).\
    coalesce(1).\
    toDF().write.format("csv").mode('overwrite').save(path_output_avgRatPerGenre)

The first one is definetely better!

### Pay attention

In the pre-aggregation pattern, be careful not doing the "average of the average", but computing the average only at the end - otherwise, the results won't match.

However, it really depends on what you want to achieve: for a given genre, assume that you have 1 movie with 1000 good ratings (rated 5) and 100 movies with bad ratings (rated 1). What should the average rating of the genre be?
- If popular movies should weigh more, you should just do a single average on all the ratings: the result should then be (1000\*5 + 100\*1) / (1000 + 100) = 4.64.
- If all movies should weigh the same, then you should do the "average of the average": the first movie has an average rating of 5, all others an average rating of 1, so it becomes (5\*1 + 100\*1) / 101 = 1.04.

Results can vary significantly; think about it when doing statistical analyses!